In [1]:
import numpy as np
     


In [2]:

def normalize(state, verbose=False):
    norm = np.linalg.norm(state)
    if verbose:
      print(f"[normalize] norm = {norm}")
    return state / norm if norm > 0 else state

In [3]:

v = np.array([3+4j, 0])
print("Before normalize:", v)
nv = normalize(v)
print("After normalize:", nv)
print("Norm:", np.linalg.norm(nv))  # Expected: 1.0

Before normalize: [3.+4.j 0.+0.j]
After normalize: [0.6+0.8j 0. +0.j ]
Norm: 1.0


In [4]:
def int_to_bitstring(x, length, verbose=False):
    bits = [int(b) for b in bin(x)[2:].zfill(length)]
    if verbose:
      print(f"[int_to_bitstring] x={x}, length={length} -> bits={bits}")
    return bits
     

In [5]:

print(int_to_bitstring(5, 4))  # Expected: [0, 1, 0, 1] (binary 0101)
print(int_to_bitstring(2, 3))  # Expected: [0, 1, 0] (binary 010)
     

[0, 1, 0, 1]
[0, 1, 0]


In [6]:
def extract_single_qubit_state(full_state, qubit_index, n, verbose=False):
    dim = 2 ** n
    amp_vec = np.zeros(2, dtype=complex)
    if verbose:
      print(f"[extract_single_qubit_state] Extracting qubit {qubit_index} from full state of dim {dim}")
    for i in range(dim):
        bits = int_to_bitstring(i, n)
        amp_vec[bits[qubit_index]] += full_state[i]
    norm = np.linalg.norm(amp_vec)
    if verbose:
      print(f"[extract_single_qubit_state] amp_vec before normalization: {amp_vec}, norm={norm}")
    if norm == 0:
        if verbose:
          print("[extract_single_qubit_state] WARNING: zero norm vector!")
        return amp_vec
    normalized = amp_vec / norm
    if verbose:
      print(f"[extract_single_qubit_state] normalized single-qubit vector: {normalized}")
    return normalized
     

In [7]:

full_state = normalize(np.array([1, 0, 0, 1]))
print("Full state:", full_state)
q0 = extract_single_qubit_state(full_state, 0, 2)
print("Qubit 0 vector:", q0)  # Expected roughly [1/sqrt(2), 1/sqrt(2)] up to phase
q1 = extract_single_qubit_state(full_state, 1, 2)
print("Qubit 1 vector:", q1)  # Expected roughly [1/sqrt(2), 1/sqrt(2)] up to phase

Full state: [0.70710678 0.         0.         0.70710678]
Qubit 0 vector: [0.70710678+0.j 0.70710678+0.j]
Qubit 1 vector: [0.70710678+0.j 0.70710678+0.j]


In [8]:

def measure_in_basis(state, basis, verbose=False):
    if verbose:
      print(f"[measure_in_basis] Measuring state (dim={len(state)}) in basis with {len(basis)} vectors")
    for i, b in enumerate(basis):
        if verbose:
          print(f"  Basis vector {i}: norm={np.linalg.norm(b):.4f}, first entries={b[:5]}")
    probs = []
    for i, b in enumerate(basis):
        overlap = np.vdot(b, state)
        prob = np.abs(overlap)**2
        if verbose:
          print(f"  Overlap with basis vector {i}: {overlap}, prob={prob}")
        probs.append(prob)
    probs = np.array(probs)
    total_prob = probs.sum()
    if verbose:
      print(f"  Total probability sum = {total_prob}")
    if total_prob == 0:
        if verbose:
          print("[measure_in_basis] ERROR: total probability zero, cannot sample")
        return None, 0
    probs /= total_prob
    outcome = np.random.choice(len(basis), p=probs)
    if verbose:
      print(f"  Selected outcome: {outcome} with probability {probs[outcome]:.4f}")
    return outcome, probs[outcome]
     

In [9]:

plus = normalize(np.array([1, 1]))
zero = np.array([1, 0])
one = np.array([0, 1])
basis = [zero, one]
print("Measuring |+> in computational basis (|0>,|1>):")
for _ in range(5):
    outcome, prob = measure_in_basis(plus, basis)
    print(f"Outcome: {outcome}, Probability: {prob:.3f}")
# Expected outcomes: roughly balanced 0s and 1s with ~0.5 probability each

Measuring |+> in computational basis (|0>,|1>):
Outcome: 0, Probability: 0.500
Outcome: 1, Probability: 0.500
Outcome: 1, Probability: 0.500
Outcome: 0, Probability: 0.500
Outcome: 1, Probability: 0.500


In [10]:
def get_reduced_state(state_vector, measured_qubits, outcomes, verbose=False):
    if verbose:
      print(f"[get_reduced_state] Measuring qubits {measured_qubits} with outcomes {outcomes}")
    psi = state_vector.copy()
    dim = len(psi)
    n_qubits = int(np.log2(dim))
    for qb, outcome in zip(measured_qubits, outcomes):
        if verbose:
          print(f"  Projecting on qubit {qb} = {outcome}")
        new_psi = np.zeros_like(psi)
        for i in range(dim):
            bits = int_to_bitstring(i, n_qubits)
            if bits[qb] == outcome:
                new_psi[i] = psi[i]
        norm = np.linalg.norm(new_psi)
        if verbose:
          print(f"  Norm after projection: {norm}")
        if norm == 0:
            if verbose:
              print("  ERROR: Measurement outcome impossible!")
            return None
        psi = new_psi / norm
        if verbose:
          print(f"  State after projection (first 5 amplitudes): {psi[:5]}")
    return psi
     

In [11]:

state = normalize(np.array([1, 0, 0, 1]))
reduced = get_reduced_state(state, [0], [0])
print("Reduced state after measuring qubit 0=0:", reduced)
# Expected: State close to |00> => first amplitude ~1, others ~0

Reduced state after measuring qubit 0=0: [1. 0. 0. 0.]


In [12]:

def DT_basis_for_x(x, verbose=False):
    if len(x) == 0:
        plus = normalize(np.array([1, 1]))
        minus = normalize(np.array([1, -1]))
        if verbose:
          print(f"[DT_basis_for_x] x = {x}: Using Hadamard basis for last qubit")
        return [plus, minus]
    else:
        if verbose:
          print(f"[DT_basis_for_x] x = {x}: No last qubits to measure")
        return []
     


In [13]:

basis_empty = DT_basis_for_x([])
print("Basis for empty x:", basis_empty)
# Expected: Hadamard basis vectors ~ [ [1,1]/√2, [1,-1]/√2 ]

basis_nonempty = DT_basis_for_x([0])
print("Basis for x=[0]:", basis_nonempty)
# Expected: empty list []
     

Basis for empty x: [array([0.70710678, 0.70710678]), array([ 0.70710678, -0.70710678])]
Basis for x=[0]: []


In [14]:
def certify(lab_state, hyp_state, verbose=False):
    n = int(np.log2(len(hyp_state)))
    if verbose:
        print(f"[certify] Starting certification for {n}-qubit state")

    k = np.random.randint(1, n + 1)
    if verbose:
        print(f"[certify] Random k chosen: {k}")

    # Step 2: Measure first k-1 qubits in computational basis
    if k == 1:
        x = []
        psi = lab_state
        if verbose:
            print("[certify] No first qubits to measure (k=1)")
    else:
        measured_qubits = list(range(k - 1))
        psi = lab_state
        outcomes = []
        if verbose:
            print(f"[certify] Measuring first {k - 1} qubits in computational basis")
        for qb in measured_qubits:
            p0 = 0.0
            p1 = 0.0
            dim = len(psi)
            n_qubits = int(np.log2(dim))
            for i in range(dim):
                bits = int_to_bitstring(i, n_qubits)
                amp = psi[i]
                if bits[qb] == 0:
                    p0 += np.abs(amp)**2
                else:
                    p1 += np.abs(amp)**2
            if verbose:
                print(f"  Qubit {qb}: P(0)={p0:.4f}, P(1)={p1:.4f}")
            outcome = np.random.choice([0, 1], p=[p0, p1])
            if verbose:
                print(f"  Sampled outcome: {outcome}")
            outcomes.append(outcome)
            psi = get_reduced_state(psi, [qb], [outcome], verbose)
            if psi is None:
                if verbose:
                    print("[certify] Measurement outcome impossible, rejecting.")
                return "Reject"
        x = outcomes

    if verbose:
        print(f"[certify] After measuring first {k - 1} qubits, outcome x = {x}")
        print(f"[certify] State after measurement (first 5 amplitudes): {psi[:5]}")

    # Step 3: Measure last n-k qubits in basis Tx
    last_n_minus_k = n - k
    Tx = DT_basis_for_x(x)
    if last_n_minus_k > 0:
        if verbose:
            print(f"[certify] Measuring last {last_n_minus_k} qubits in DT basis")
        outcome_ell, prob_ell = measure_in_basis(psi, Tx, verbose)
        if outcome_ell is None:
            if verbose:
                print("[certify] Measurement in DT basis failed, rejecting.")
            return "Reject"
        if verbose:
            print(f"[certify] Outcome ℓ = {outcome_ell}, probability = {prob_ell}")
        psi = Tx[outcome_ell]
        if verbose:
            print(f"[certify] State after measuring last n-k qubits (first 5 amplitudes): {psi[:5]}")
    else:
        outcome_ell = None
        if verbose:
            print("[certify] No last qubits to measure in DT basis")

    # Step 4: Measure kth qubit in basis containing |hyp'>
    measured_qubits_final = list(range(k - 1))
    outcomes_final = x.copy()
    if outcome_ell is not None:
        measured_qubits_final.append(k)
        outcomes_final.append(outcome_ell)

    hyp_cond = get_reduced_state(hyp_state, measured_qubits_final, outcomes_final, verbose)
    if hyp_cond is None:
        if verbose:
            print("[certify] Hyp conditioned state impossible, rejecting.")
        return "Reject"

    if verbose:
        print(f"[certify] Hyp conditioned state |hyp'> (first 5 amplitudes): {hyp_cond[:5]}")

    hyp_prime_1q = extract_single_qubit_state(hyp_cond, k - 1, n, verbose)
    single_qubit_state = extract_single_qubit_state(psi, k - 1, n, verbose)

    orth_vec = np.array([-hyp_prime_1q[1].conj(), hyp_prime_1q[0].conj()])
    basis_k = [hyp_prime_1q, orth_vec]

    if verbose:
        print(f"[certify] Measurement basis for kth qubit:")
        for i, b in enumerate(basis_k):
            print(f"  Basis {i} (norm {np.linalg.norm(b):.4f}): {b}")

    accept, prob_accept = measure_in_basis(single_qubit_state, basis_k, verbose)
    if accept is None:
        if verbose:
            print("[certify] Final measurement failed, rejecting.")
        return "Reject"
    if verbose:
        print(f"[certify] Final measurement outcome: {accept}, probability: {prob_accept:.4f}")

    return "Accept" if accept == 0 else "Reject"
     

In [15]:

# Test 1
n = 2
hyp = normalize(np.array([1, 0, 0, 1], dtype=complex))  # Bell state
lab = hyp.copy()  # Identical state
decision = certify(lab, hyp, True)
print("Decision:", decision)

[certify] Starting certification for 2-qubit state
[certify] Random k chosen: 2
[certify] Measuring first 1 qubits in computational basis
  Qubit 0: P(0)=0.5000, P(1)=0.5000
  Sampled outcome: 0
[get_reduced_state] Measuring qubits [0] with outcomes [np.int64(0)]
  Projecting on qubit 0 = 0
  Norm after projection: 0.7071067811865475
  State after projection (first 5 amplitudes): [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[certify] After measuring first 1 qubits, outcome x = [np.int64(0)]
[certify] State after measurement (first 5 amplitudes): [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[certify] No last qubits to measure in DT basis
[get_reduced_state] Measuring qubits [0] with outcomes [np.int64(0)]
  Projecting on qubit 0 = 0
  Norm after projection: 0.7071067811865475
  State after projection (first 5 amplitudes): [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[certify] Hyp conditioned state |hyp'> (first 5 amplitudes): [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[extract_single_qubit_state] Extracting qubit 1 from full state of dim 4
[ex

In [17]:
# Test 2
n = 2
hyp = normalize(np.array([1, 0, 0, 1], dtype=complex))  # Bell state
lab = normalize(np.array([0, 1, 1, 0], dtype=complex))  # Orthogonal
decision = certify(lab, hyp)
print("Decision:", decision)

Decision: Reject


In [18]:
# Test 3
n = 2
hyp = normalize(np.array([1, 0, 0, 1], dtype=complex))
lab = normalize(np.array([0.95, 0, 0, 0.31], dtype=complex))
decision = certify(lab, hyp)
print("Decision:", decision)

Decision: Accept


In [20]:
import numpy as np

# -----------------------------
# Utility: Normalize a state
# -----------------------------
def normalize(state):
    return state / np.linalg.norm(state)

# -----------------------------
# Measure state in given basis
# -----------------------------
def measure_in_basis(state, basis, verbose=True):
    dim = len(state)
    
    # Dimension check (prevents your previous error)
    for b in basis:
        if len(b) != dim:
            raise ValueError("Basis vector dimension does not match state dimension.")
    
    probs = []
    for i, b in enumerate(basis):
        overlap = np.vdot(b, state)
        prob = np.abs(overlap) ** 2
        probs.append(prob)
        
        if verbose:
            print(f"Outcome {i}: Probability = {prob:.4f}")
    
    # Return most likely outcome
    max_index = np.argmax(probs)
    return max_index, probs[max_index]

# -----------------------------
# Certification function
# -----------------------------
def certify(lab_state, hyp_state, epsilon=0.1, verbose=True):
    threshold = 1 - epsilon
    
    # Fidelity = |<hyp|lab>|^2
    fidelity = np.abs(np.vdot(hyp_state, lab_state)) ** 2
    
    if verbose:
        print(f"Fidelity = {fidelity:.4f}")
        print(f"Threshold = {threshold:.4f}")
    
    if fidelity >= threshold:
        return "ACCEPT"
    else:
        return "REJECT"

# -----------------------------
# 2-Qubit Computational Basis
# -----------------------------
basis_2q = [
    np.array([1,0,0,0], dtype=complex),  # |00>
    np.array([0,1,0,0], dtype=complex),  # |01>
    np.array([0,0,1,0], dtype=complex),  # |10>
    np.array([0,0,0,1], dtype=complex)   # |11>
]

# -----------------------------
# Test 4
# -----------------------------
n = 2
epsilon = 0.1

# Hypothesis state (Bell-like)
hyp = normalize(np.array([1, 0, 0, 1], dtype=complex))

# Construct lab state so fidelity ≈ 0.89 < 0.9
lab = normalize(hyp * np.sqrt(0.89) + np.array([0.1, 0.1, 0.1, 0.1]))

# Optional: measurement demonstration
print("Measurement Probabilities:")
measure_in_basis(lab, basis_2q)

# Certification
decision = certify(lab, hyp, epsilon)
print("\nDecision:", decision)


Measurement Probabilities:
Outcome 0: Probability = 0.4916
Outcome 1: Probability = 0.0084
Outcome 2: Probability = 0.0084
Outcome 3: Probability = 0.4916
Fidelity = 0.9833
Threshold = 0.9000

Decision: ACCEPT


In [21]:

# Test 5
n = 2
hyp = normalize(np.kron([1, 0], [1, 0]))  # |00⟩
lab = normalize(np.kron([0.99, 0.1], [1, 0]))  # ≈ |00⟩
decision = certify(lab, hyp)
print("Decision:", decision)
     

Fidelity = 0.9899
Threshold = 0.9000
Decision: ACCEPT


In [22]:
# Test 7
n = 2
hyp = normalize(np.array([1, 0, 0, 1], dtype=complex))
lab = normalize(np.array([1, 0, 0, 0], dtype=complex))  # Only matches half
decision = certify(lab, hyp)
print("Decision:", decision)
     

Fidelity = 0.5000
Threshold = 0.9000
Decision: REJECT


In [23]:
# Test 6
n = 2
hyp = normalize(np.array([1, 0, 0, 1], dtype=complex))  # Bell
lab = normalize(np.kron([1, 0], [0, 1]))  # |01⟩ (product state)
decision = certify(lab, hyp)
print("Decision:", decision)

Fidelity = 0.0000
Threshold = 0.9000
Decision: REJECT


In [24]:
# Test 8
n = 4
hyp = normalize(np.kron(np.array([1, 0, 0, 1]), np.array([1, 0, 0, 1])))
lab = normalize(hyp + 0.01 * np.random.randn(16))  # Very small perturbation
decision = certify(lab, hyp)
print("Decision:", decision)

Fidelity = 0.9984
Threshold = 0.9000
Decision: ACCEPT
